## INTO THE DEEP LEARNING UNIVERSE

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import numpy as np


In [2]:
train_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor(),
    )


In [3]:
test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor(),
    )

In [4]:
batch_size = 64

In [5]:
train_loader = DataLoader(train_data, batch_size = batch_size)
test_loader = DataLoader(test_data, batch_size = batch_size)

In [6]:
for X, y in test_loader:
    print(f"Shape of X: {X.shape}")
    print(f"Shape of y: {y.shape}")
    break


Shape of X: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64])


In [7]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [8]:
print(f"Using {device}")

Using cpu


In [9]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
            )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


In [10]:
model = NeuralNet().to(device)

In [11]:
print(model)


NeuralNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [12]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)


In [13]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch%100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")


In [14]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        test_loss /= num_batches
        correct /= size
        print(f"test error: \n Acc: {100*correct:>0.1f}, avg loss: {test_loss:>8f}\n")



In [15]:
epoch = 5

In [16]:
for t in range(epoch):
    print(f"Epoch{t + 1} \n ----------------------------")
    train(train_loader, model, loss_fn, optimizer)
    test(test_loader, model, loss_fn)
print("Done")

Epoch1 
 ----------------------------
loss: 2.306738 [   64/60000]
loss: 2.293072 [ 6464/60000]
loss: 2.266226 [12864/60000]
loss: 2.267465 [19264/60000]
loss: 2.242602 [25664/60000]
loss: 2.217076 [32064/60000]
loss: 2.230056 [38464/60000]
loss: 2.187651 [44864/60000]
loss: 2.199427 [51264/60000]
loss: 2.165623 [57664/60000]
test error: 
 Acc: 44.2, avg loss: 2.156245

Epoch2 
 ----------------------------
loss: 2.168254 [   64/60000]
loss: 2.163271 [ 6464/60000]
loss: 2.097800 [12864/60000]
loss: 2.121795 [19264/60000]
loss: 2.071419 [25664/60000]
loss: 2.009935 [32064/60000]
loss: 2.049729 [38464/60000]
loss: 1.961364 [44864/60000]
loss: 1.979541 [51264/60000]
loss: 1.912710 [57664/60000]
test error: 
 Acc: 52.4, avg loss: 1.902170

Epoch3 
 ----------------------------
loss: 1.937667 [   64/60000]
loss: 1.912861 [ 6464/60000]
loss: 1.789343 [12864/60000]
loss: 1.837205 [19264/60000]
loss: 1.727654 [25664/60000]
loss: 1.677004 [32064/60000]
loss: 1.712772 [38464/60000]
loss: 1.59785